In [ ]:
from datetime import datetime, timedelta
from enum import IntEnum
from math import ceil
import numpy as np
import pickle
from PySeirCampus import *
import random

# Functions to create Contact Structure.

In [ ]:
# Student and faculty generation is unchanged from above.
def make_population(semester):
    # Students.
    for year in [1, 2, 3, 4]:
        offset = (year - 1) * 4000
        for i in range(1200):
            semester.students[offset + i] = {'type':'s', 'year':year, 'field': 'Business', 'social':'high'}
        for i in range(1350):
            semester.students[offset + i + 1200] = {'type':'s', 'year':year, 'field':'Business', 'social':'medium'}
        for i in range(450):
            semester.students[offset + i + 2550] = {'type':'s', 'year':year, 'field':'Business', 'social':'low'}
        for i in range(240):
            semester.students[offset + i + 3000] = {'type':'s', 'year':year, 'field':'STEM', 'social':'high'}
        for i in range(270):
            semester.students[offset + i + 3240] = {'type':'s', 'year':year, 'field':'STEM', 'social':'medium'}
        for i in range(90):
            semester.students[offset + i + 3510] = {'type':'s', 'year':year, 'field':'STEM', 'social':'low'}
        for i in range(160):
            semester.students[offset + i + 3600] = {'type':'s', 'year':year, 'field':'Humanities', 'social':'high'}
        for i in range(180):
            semester.students[offset + i + 3760] = {'type':'s', 'year':year, 'field':'Humanities', 'social':'medium'}
        for i in range(60):
            semester.students[offset + i + 3940] = {'type':'s', 'year':year, 'field':'Humanities', 'social':'low'}
    
    # Faculty.
    for i in range(600):
        semester.students[16000 + i] = {'type':'f', 'field':'Business'}
    for i in range(120):
        semester.students[16600 + i] = {'type':'f', 'field':'STEM'}
    for i in range(80):
        semester.students[16720 + i] = {'type':'f', 'field':'Humanities'}

In [ ]:
def make_courses(semester):
    class Area(IntEnum):
        GENERAL = 4
        BUSINESS = 3
        STEM = 2
        HUMANITIES = 1
    
    Sizes = [10, 20, 30, 40, 50, 75, 150]
    SizeDistributions = {Area.GENERAL: [8, 40, 140, 60, 30, 40, 10],
                         Area.BUSINESS: [24, 120, 420, 180, 90, 120, 30],
                         Area.STEM: [5, 24, 84, 36, 18, 24, 6],
                         Area.HUMANITIES: [3, 16, 56, 24, 12, 16, 4]}
    
    S = {a : [size for size, count in zip(Sizes, dist) for i in range(count)]
         for a, dist in SizeDistributions.items()}
    Tx = {a : sum(counts) for a, counts in S.items()}
    Px = {a : [size / Tx[a] for size in classes] for a, classes in S.items()}
    Cx = {a : len(Px[a]) for a in Px}
    
    def Y(area):
        if random.random() < 0.2:
            area = Area.GENERAL
        return (area, np.random.choice(Cx[area], p = Px[area]))
    
    def GroupDraw(area):
        ''' Note format (area, index) has area before general, smaller classes first. '''
        return sorted([Y(area) for i in range(4)])
    
    def make_area(area, count):
        students = {y: [] for y in range(4)}
        for i in range(int(count // 4)):
            for y, assignment in zip(students, GroupDraw(area)):
                students[y].append(assignment)
        return students
    
    translate = {'Business': Area.BUSINESS,
                 'STEM': Area.STEM,
                 'Humanities': Area.HUMANITIES}
    
    rosters = defaultdict(list)
    
    for i in range(4):  # Assign each student 4 times.
        pool = defaultdict(list)
        for s, data in semester.students.items():
            if data['type'] == 's':
                pool[data['year'], translate[data['field']]].append(s)
        for key in pool:
            np.random.shuffle(pool[key])

        # Now we are ready to make the class assignments!
        assignments = {Area.BUSINESS: make_area(Area.BUSINESS, 12000),
                       Area.STEM: make_area(Area.STEM, 2400),
                       Area.HUMANITIES: make_area(Area.HUMANITIES, 1600)}

        for (y, f), ss in pool.items():
            for s, a in zip(ss, assignments[f][y - 1]):
                rosters[a].append(s)
    
    # Also must assign the teachers!
    byarea = defaultdict(list)
    for area, index in rosters:
        byarea[area].append(index)
    for s in range(16000, 16800):
        field = translate[semester.students[s]['field']]
        classes = np.random.choice(byarea[field], size = 2, replace = False)
        for c in classes:
            rosters[field, c].append(s)
    
    # Now write the classes to the semester object.
    for i, ((area, index), roster) in enumerate(rosters.items()):
        days = 'MW' if i % 2 else 'TR'
        duration = 10 / len(roster) # Was 100 / len(---)
        meetinfo = [('9/6/2021', '12/17/2021', days, duration)]
        name = str((area, i))
        info = {'name': name, 'type':'Classroom'}
        semester.add_meeting(name, info, meetinfo, roster)

In [ ]:
def make_clubs(semester):
    ''' This version of clubs is only different in the duration of the meets. '''
    
    # General clubs.  50.  Each student join one with probability 1/5.
    general_club_choice = np.random.choice(list(range(50)), size = 16000)
    general_club_join = np.random.choice([0, 1], size = 16000, p = [0.8, .2])
    general_club_members = defaultdict(list)
    for i, (club, join) in enumerate(zip(general_club_choice, general_club_join)):
        if join:
            general_club_members[club].append(i)
    for i in range(50):
        name = 'GeneralClub' + str(i)
        info = {'name': name, 'type': 'Club'}
        meetinfo = [('9/6/2021', '12/17/2021', 'R', 100 / len(general_club_members[i]))]
        semester.add_meeting(name, info, meetinfo, general_club_members[i])
    
    # Field specific clubs.
    fields = {'Business':30, 'STEM':20, 'Humanities':10}
    student_fields = {f : [] for f in fields}
    for i in range(16000):
        student_fields[semester.students[i]['field']].append(i)
    for f, count in fields.items():
        club_choice = np.random.choice(list(range(count)), size = len(student_fields[f]))
        club_join = np.random.choice([0, 1], size = len(student_fields[f]), p = [.8, .2])
        members = defaultdict(list)
        for i, (club, join) in enumerate(zip(club_choice, club_join)):
            if join:
                members[club].append(i)
        for club in range(count):
            name = f + 'Club' + str(club)
            info = {'name':name, 'type': 'Club'}
            meetinfo = [('9/6/2021', '12/17/2021', 'R', 100 / len(members[club]))]
            semester.add_meeting(name, info, meetinfo, members[club])

In [ ]:
def make_socialization(semester, count_small, count_large, S, partying = False):
    ''' This version of '''
    fields = ['Business', 'STEM', 'Humanities']
    years = [1, 2, 3, 4]
    categories = [(f, y) for f in fields for y in years]
    def get_field():
        return random.choices(fields, weights=[.75, .15, .1])[0]
    
    # Categorize students by field, year, and socialization level.
    students = defaultdict(lambda : defaultdict(list))
    for i, info in semester.students.items():
        if i < 16000:
            y = info['year']
            f = info['field']
            s = info['social']
            students[(f, y)][s].append(i)
    
    def make(p_continue = 0.5):
        f = get_field()
        y = random.choice(years)
        members = [(f, y)]
        while random.random() <= p_continue:
            new = random.random()
            if new <= 1/6:
                f, y = random.choice(fields), random.choice(years)
            elif new <= 2/6:
                f = get_field()
            elif new <= 1/2:
                y = random.choice(years)
            else:
                pass
            members.append((f, y))
        return members
    
    social_levels = ['low', 'medium', 'high']
    social_weights = [.1, .3, .6]
    def convert(members):
        realized_members = []
        levels = random.choices(social_levels, weights = social_weights, k = len(members))
        for m, l in zip(members, levels):
            #print(m, l)
            #print(list(students[m].keys()))
            realized_members.append(random.choice(students[m][l]))
        return realized_members
    
    i = [] # Globals weren't working.
    def place(group, days):
        meetinfo = [('9/6/2021', '12/17/2021', days, S * 100 / len(group))]
        name = 'SocialGroup' + str(len(i))
        info = {'name':name, 'type': 'Socialization'}
        semester.add_meeting(name, info, meetinfo, convert(group))
        i.append(0)
    
    def place_small(group):
        days = []
        for day in ['M', 'T', 'W', 'R']:
            if random.random() > 0.5:
                days.append(day)
        if len(days):
            place(group, ''.join(days))
    
    def place_large(group):
        days = []
        for day in ['R', 'F']:
            if random.random() > 0.5:
                days.append(day)
        if len(days):
            place(group, ''.join(days))
        
    
    list(map(place_large, [make(0.9) for x in range(count_large)]))
    list(map(place_small, [make(0.5) for x in range(count_small)]))
    
    if partying:
        date = datetime.strptime('9/6/2021', '%m/%d/%Y')
        enddate = datetime.strptime('12/17/2021', '%m/%d/%Y')
        while date <= enddate:
            if date.weekday() == 4:
                #The base model should have occasional parties. 
                #Please add for every Friday an event that combines 
                #5 randomly selected large social groups into a new 
                #meeting. Take the duration to be S*100 / L minutes 
                #with S=20 the party factor and L the total number 
                #of people in the meeting. 
                groups = np.random.choice(list(range(count_large)),
                                          size = 5,
                                          replace = False)
                members = set()
                for g in groups:
                    name = 'SocialGroup' + str(g)
                    members.update(semester.meeting_enrollment[name])
                date_string = date.strftime('%m/%d/%Y')
                meetinfo = [(date_string, date_string, 'F', 20 * 100 / len(members))]
                name = 'SocialGroupParty' + date_string
                info = {'name': name, 'type': 'Socialization'}
                semester.add_meeting(name, info, meetinfo, members)
            date += timedelta(days = 1)
    

In [ ]:
def make_broad(semester):
    ''' This version updates times and meeting dates. '''
    # All agents spend 20 minutes per week meeting with students/faculty in their area.
    by_area = defaultdict(list)
    for s, info in semester.students.items():
        by_area[info['field']].append(s)
    for area, members in by_area.items():
        name = 'Broad' + area
        info = {'name':name, 'type':'Broad'}
        meetinfo = [('9/6/2021', '12/17/2021', 'MTWR', 20 / 4 / len(members))]
        semester.add_meeting(name, info, meetinfo, members)
    
    # All agents spend 10 minutes per week meeting with all agents in the model.
    name = 'BroadAmbient'
    info = {'name':name, 'type':'Broad'}
    meetinfo = [('9/6/2021', '12/17/2021', 'MTWR', 10 / 16800)]
    semester.add_meeting(name, info, meetinfo, semester.students)

In [ ]:
def make_residence_hall(semester):
    # Get list of 1st and 2nd year students.
    candidates = []
    for s, data in semester.students.items():
        if data['type'] == 's' and data['year'] in [1, 2]:
            candidates.append(s)
    residents = np.random.choice(candidates, size = 400, replace = False)
    
    # Roommates spend 300 minutes per day together.
    np.random.shuffle(residents)
    meetinfo = [('9/6/2021', '12/17/2021', 'MTWRFSU', 300)]
    for i, roommates in enumerate(zip(residents[:200], residents[200:])):
        name = 'RoomMates-' + str(i)
        info = {'name': name, 'type':'Dorm'}
        semester.add_meeting(name, info, meetinfo, roommates)
    
    # All redidents get 10 minutes of incidental contact with each other.
    meetinfo = [('9/6/2021', '12/17/2021', 'MTWRFSU', 100 / 400)] # NOte: Just added denominator.
    name = 'Residents-Incidental'
    info = {'name': name, 'type':'Dorm'}
    semester.add_meeting(name, info, meetinfo, residents)

In [ ]:
def thin_semester(semester):
    # Get rid of half the students.
    for i in range(0, 16800, 2):
        semester.remove_student(i)
    
    # Recalibrate the times.
    durations = {}
    for meeting, students in semester.meeting_enrollment.items():
        name = semester.meetings[meeting]['name']
        if not len(semester.meeting_enrollment[meeting]):
            continue
        if 'Residents' in name or 'RoomMates' in name:
            continue
        elif 'BroadAmbient' in name:
            durations[meeting] = 10 / len(semester.meeting_enrollment[meeting])
        elif 'Broad' in name:
            durations[meeting] = 20 / 4 / len(semester.meeting_enrollment[meeting])
        elif 'SocialGroup' in name:
            durations[meeting] = 10 * 100 / len(semester.meeting_enrollment[meeting])
        else:
            durations[meeting] = 100 / len(semester.meeting_enrollment[meeting])
    
    # Now set the times.
    for date, meetings in semester.meeting_dates.items():
        for meeting in meetings:
            if meeting in durations:
                semester.meeting_dates[date][meeting] = durations[meeting]

In [ ]:
# Repeated from above.
def semester_generator_fully_connected(daily_average_exposure):
    def generator(semester):
        semester = Semester()
        make_population(semester)
        prorated_exposure = daily_average_exposure / 16800 * (7 / 5)
        meetinfo = [('9/6/2021', '12/17/2021', 'MTWRF', prorated_exposure)]
        name = 'FullyConnected'
        info = {'name' : name}
        semester.add_meeting(name, info, meetinfo, semester.students)
        return semester
    return generator

In [ ]:
def semester_generator(count_small, count_large):
    def generator(semester):
        semester = Semester()
        make_population(semester)
        make_courses(semester)
        make_clubs(semester)
        make_socialization(semester, count_small, count_large, 20, partying = True)
        make_broad(semester)
        make_residence_hall(semester)
        return semester
    return generator

In [ ]:
def semester_generator_noparty(count_small, count_large):
    def generator(semester):
        semester = Semester()
        make_population(semester)
        make_courses(semester)
        make_clubs(semester)
        make_socialization(semester, count_small, count_large, 20)
        make_broad(semester)
        make_residence_hall(semester)
        return semester
    return generator

In [ ]:
def semester_generator_thinned(count_small, count_large):
    def generator(semester):
        semester = Semester()
        make_population(semester)
        make_courses(semester)
        make_clubs(semester)
        make_socialization(semester, count_small, count_large, 10)
        make_broad(semester)
        make_residence_hall(semester)
        thin_semester(semester)
        return semester
    return generator

In [ ]:
def get_exposure_time(semester):
    time = 0
    for d in range(6, 13):
        day = datetime(2021, 9, d)
        for meeting, duration in semester.meeting_dates[day].items():
            time += duration * len(semester.meeting_enrollment[meeting]) * \
                    (len(semester.meeting_enrollment[meeting]) - 1)
    return time / 7 / len(semester.students)

# Functions to define experiment parameters and present results.

In [ ]:
# A vaccinated person is asymptomatic and has reduced reception/transmission.
# Antibodies protect someone completely.

class VaccineIntervention:
    def __init__(self, p_vaccinated, p_antibodies):
        self.pv = p_vaccinated
        self.pa = p_antibodies
    def testing(self, simulation, date): # Overload testing function to set vaccinations on first day.
        if date == datetime(2021, 9, 6):
            for i in simulation.semester.students:
                if random.random() <= self.pv:
                    simulation.state.V.add(i)
                if random.random() <= self.pa:
                    simulation.state.S.discard(i)
                    simulation.state.A.add(i)
    def __str__(self):
        return 'VaccineIntervention(Protection level' + str(self.pp) + ')'

In [ ]:
from matplotlib.pyplot import *

def boxplot_totals(scenario_distributions, save = None):
    scenario_totals = defaultdict(list)
    for scenario in scenario_distributions:
        for counts in scenario_distributions[scenario].values():
            items = len(counts)
            break
        for i in range(items):
            total = 0
            for counts in scenario_distributions[scenario].values():
                total += counts[i]
            scenario_totals[scenario].append(total)
    
    xs = []
    ys = []
    for scenario in sorted(scenario_totals):
        xs.append(scenario)
        ys.append(scenario_totals[scenario])
    
    boxplot(ys)
    xticks(np.array(range(len(xs))) + 1, xs, rotation = 30)
    ylabel('Number of People')
    title('Distribution of Infection Counts by Scenario')
    tight_layout()
    #if save:
    #    gcf().savefig(save + '-counts.pdf')
    show()

def boxplot_community(scenario_distributions, save = None):
    scenario_totals = defaultdict(list)
    for scenario in scenario_distributions:
        scenario_totals[scenario] = scenario_distributions[scenario]['Community']
    
    xs = []
    ys = []
    for scenario in sorted(scenario_totals):
        xs.append(scenario)
        ys.append(scenario_totals[scenario])
    
    boxplot(ys)
    xticks(np.array(range(len(xs))) + 1, xs, rotation = 30)
    ylabel('Number of People')
    title('Community Source Infection Counts by Scenario')
    tight_layout()
    #if save:
    #    gcf().savefig(save + '-community_sources.pdf')
    show()

def stackedbar_sources(scenario_sources, plottitle, save = None):
    source_list = set()
    for ss in scenario_sources.values():
        source_list.update(set(ss))
    
    source_vectors = defaultdict(list)
    for name, ss in scenario_sources.items():
        for s, value in ss.items():
            source_vectors[s].append(value)
    
    at_level = [0 for x in range(len(scenario_sources))]
    for source in sorted(source_list):
        bar(list(scenario_sources.keys()), source_vectors[source], .5, label = source, bottom = at_level)
        for i, x in enumerate(source_vectors[source]):
            at_level[i] += x

    legend()
    xticks(rotation = 30)
    ylabel('Number of People')
    title(plottitle)
    tight_layout()
    #if save:
    #    gcf().savefig(save + '-sources.pdf')
    show()

In [ ]:
repetitions = 100

def parameters_scenario1(): # Baseline case.
    parameters = Parameters()
    parameters.rate = rate
    parameters.daily_spontaneous_prob = 2 / (7 * 16800)
    parameters.infection_duration = VariedResponse(1 / 3.0, 1 / 5.0, 1 / 2.0, 0.5)
    parameters.preclass_interaction_time = 0
    parameters.initial_exposure = 10
    parameters.intervention_policy = VaccineIntervention(0.0, 0.2) # (vax, antibodies)
    parameters.preprocess = semester_generator(3000, 300)
    parameters.start_date = datetime(2021, 9, 6)
    parameters.end_date = datetime(2021, 12, 17)
    parameters.repetitions = repetitions
    parameters.vaccine_benefit_self = 0.9 # (Default value)
    parameters.vaccine_benefit_others = 0.5 # (Default value)
    return parameters

def parameters_scenario2(): #_ReducedInPerson():
    parameters = Parameters()
    parameters.rate = rate
    parameters.daily_spontaneous_prob = 2 / (7 * 16800)
    parameters.infection_duration = VariedResponse(1 / 3.0, 1 / 5.0, 1 / 2.0, 0.5)
    parameters.preclass_interaction_time = 0
    parameters.initial_exposure = 10
    parameters.intervention_policy = VaccineIntervention(0.0, 0.2) # 1.0, 0.1)
    parameters.preprocess = semester_generator_noparty(1500, 150)
    parameters.start_date = datetime(2021, 9, 6)
    parameters.end_date = datetime(2021, 12, 17)
    parameters.repetitions = repetitions
    return parameters

def parameters_scenario3(v, ri, ro): # Baseline case.
    parameters = Parameters()
    parameters.rate = rate
    parameters.daily_spontaneous_prob = 2 / (7 * 16800)
    parameters.infection_duration = VariedResponse(1 / 3.0, 1 / 5.0, 1 / 2.0, 0.5)
    parameters.preclass_interaction_time = 0
    parameters.initial_exposure = 10
    parameters.intervention_policy = VaccineIntervention(v, 0.2) # (vax, antibodies)
    parameters.preprocess = semester_generator(3000, 300)
    parameters.start_date = datetime(2021, 9, 6)
    parameters.end_date = datetime(2021, 12, 17)
    parameters.repetitions = repetitions
    parameters.vaccine_benefit_self = ri # (Default value)
    parameters.vaccine_benefit_others = ro # (Default value)
    return parameters

def parameters_scenario4(v, ri, ro): #_ReducedInPerson():
    parameters = Parameters()
    parameters.rate = rate
    parameters.daily_spontaneous_prob = 2 / (7 * 16800)
    parameters.infection_duration = VariedResponse(1 / 3.0, 1 / 5.0, 1 / 2.0, 0.5)
    parameters.preclass_interaction_time = 0
    parameters.initial_exposure = 10
    parameters.intervention_policy = VaccineIntervention(v, 0.2) # 1.0, 0.1)
    parameters.preprocess = semester_generator_noparty(1500, 150)
    parameters.start_date = datetime(2021, 9, 6)
    parameters.end_date = datetime(2021, 12, 17)
    parameters.repetitions = repetitions
    parameters.vaccine_benefit_self = ri
    parameters.vaccine_benefit_others = ro
    return parameters

# Calibrate and execute scenarios.

In [ ]:
# Calibration: Determine a value for infectionrate based on hypothetical R_0.

'''
r_0 = 3
sample_semester = semester_generator(3000, 300)(None)
avg_daily_exposure_per_person = get_exposure_time(sample_semester)
avg_infectious_days = 5

rate = r_0 / (avg_daily_exposure_per_person * avg_infectious_days)'''

rate = 0.0032274494237379487

print('Rate computed from sample was', rate)

In [ ]:
# Scenario 1.  No vaccinations, high socialization.  Default R0.
output = run_repetitions(None, parameters_scenario1(), conciseview = True, save = 'figures/scenario1', report = True)
pickle.dump(output, open('pickles/scenario1_r03.p', 'wb'))

In [ ]:
# Scenario 1b: R0 = 2.
base_rate = rate
rate *= 2/3
output = run_repetitions(None, parameters_scenario1(), save = 'figures/scenario1-R0=2', conciseview = True, report=True)
pickle.dump(output, open('pickles/scenario1_r02.p', 'wb'))
rate = base_rate

In [ ]:
# Scenario 1c: Ro = 4.
base_rate = rate
rate *= 4/3
output = run_repetitions(None, parameters_scenario1(), save = 'figures/scenario1-R0=4', conciseview = True, report=True)
pickle.dump(output, open('pickles/scenario1_r04.p', 'wb'))
rate = base_rate

In [ ]:
# Scenario 2.  No vaccinations, lower socialization.
output = run_repetitions(None, parameters_scenario2(), save = 'figures/scenario2', conciseview = True, report=True)
pickle.dump(output, open('pickles/scenario2.p', 'wb'))

In [ ]:
# Scenario 3.  Various vaccination levels and effectiveness, high socialization.
scenario_sources = {}
scenario_distributions = {}
scenario_types = {}
for v in [.5, .75]:
    for ri, ro in [(.3, .3), (.3, .7), (.7, .3), (.7, .7)]:
            name = 'V' + str(v) + ',' + str((ri, ro))
            sources, distribution, types, summary = run_repetitions(
                    None,
                    parameters_scenario3(v, ri, ro),
                    report = True,
                    graphics = False)
            scenario_sources[name] = sources
            scenario_distributions[name] = distribution
            scenario_types[name] = types
            output = sources, distribution, types, summary
            pickle.dump(output, open('pickles/scenario3_' + name + '_r03.p', 'wb'))

boxplot_totals(scenario_distributions, save = 'figures/scenario3')
boxplot_community(scenario_distributions, save = 'figures/scenario3')
stackedbar_sources(scenario_sources, 'Source of Infections by Scenario', 
                   save = 'figures/scenario3')
stackedbar_sources(scenario_types, 'Vaccination Status of Infected Individuals by Scenario',
                   save = 'figures/scenario3-vaccine')

In [ ]:
# Scenario 3b: R0 = 2.
scenario_sources = {}
scenario_distributions = {}
scenario_types = {}
old_rate = rate
rate *= 2 / 3
for v in [.5, .75]:
    for ri, ro in [(.3, .3), (.3, .7), (.7, .3), (.7, .7)]:
            name = 'V' + str(v) + ',' + str((ri, ro))
            sources, distribution, types, summary = run_repetitions(
                    None,
                    parameters_scenario3(v, ri, ro),
                    report = True,
                    graphics = False)
            scenario_sources[name] = sources
            scenario_distributions[name] = distribution
            scenario_types[name] = types
            output = sources, distribution, types, summary
            pickle.dump(output, open('pickles/scenario3_' + name + '_r02.p', 'wb'))

rate = old_rate
boxplot_totals(scenario_distributions, save = 'figures/scenario3-R0=2')
boxplot_community(scenario_distributions, save = 'figures/scenario3-R0=2')
stackedbar_sources(scenario_sources, 'Source of Infections by Scenario', 
                   save = 'figures/scenario3-R0=2')
stackedbar_sources(scenario_types, 'Vaccination Status of Infected Individuals by Scenario',
                   save = 'figures/scenario3-R0=2-vaccine')

In [ ]:
# Scenario 3c: R0 = 4.
scenario_sources = {}
scenario_distributions = {}
scenario_types = {}
old_rate = rate
rate *= 4 / 3
for v in [.5, .75]:
    for ri, ro in [(.3, .3), (.3, .7), (.7, .3), (.7, .7)]:
            name = 'V' + str(v) + ',' + str((ri, ro))
            sources, distribution, types, summary = run_repetitions(
                    None,
                    parameters_scenario3(v, ri, ro),
                    report = True,
                    graphics = False)
            scenario_sources[name] = sources
            scenario_distributions[name] = distribution
            scenario_types[name] = types
            output = sources, distribution, types, summary
            pickle.dump(output, open('pickles/scenario3_' + name + '_r04.p', 'wb'))

rate = old_rate
boxplot_totals(scenario_distributions, save = 'figures/scenario3-R0=4')
boxplot_community(scenario_distributions, save = 'figures/scenario3-R0=4')
stackedbar_sources(scenario_sources, 'Source of Infections by Scenario', 
                   save = 'figures/scenario3-R0=4')
stackedbar_sources(scenario_types, 'Vaccination Status of Infected Individuals by Scenario',
                   save = 'figures/scenario3-R0=4-vaccine')

In [ ]:
# Scenario 4.  Various vaccination levels and effectiveness, lower socialization.
scenario_sources = {}
scenario_distributions = {}
scenario_types = {}
for v, ri, ro in [(.5, .3, .3), (.75, .7, .7)]:
    name = 'V' + str(v) + ',' + str((ri, ro))
    sources, distribution, types, summary = run_repetitions(
            None,
            parameters_scenario4(v, ri, ro),
            report = True,
            graphics = False)
    scenario_sources[name] = sources
    scenario_distributions[name] = distribution
    scenario_types[name] = types
    output = sources, distribution, types, summary
    pickle.dump(output, open('pickles/scenario4_' + name + '.p', 'wb'))

boxplot_totals(scenario_distributions, save = 'figures/scenario4')
boxplot_community(scenario_distributions, save = 'figures/scenario4')
stackedbar_sources(scenario_sources, 'Source of Infections by Scenario', 
                   save = 'figures/scenario4')
stackedbar_sources(scenario_types, 'Vaccination Status of Infected Individuals by Scenario', 
                   save = 'figures/scenario4-vaccine')

In [ ]:
# Scenario 5: Vaccination sensitivity, varying efficacy.
scenario_sources = {}
scenario_distributions = {}
scenario_types = {}
for rr in [0.2, 0.5, 0.8]:
    ri, ro = rr, rr
    for v in [.5, .6, .7, .8, .9, 1.0]:
        name = 'V' + str(v)
        sources, distribution, types, summary = run_repetitions(
                None,
                parameters_scenario3(v, ri, ro),
                report = True,
                graphics = False)
        scenario_sources[name] = sources
        scenario_distributions[name] = distribution
        scenario_types[name] = types
        output = sources, distribution, types, summary
        pickle.dump(output, open('pickles/scenario5_' + str(rr) + '_' + name + '.p', 'wb'))

    boxplot_totals(scenario_distributions, save = 'figures/scenario5_' + str(rr))
    boxplot_community(scenario_distributions, save = 'figures/scenario5_' + str(rr))
    stackedbar_sources(scenario_sources, 'Source of Infections by Scenario', 
                       save = 'figures/scenario5_' + str(rr))
    stackedbar_sources(scenario_types, 'Vaccination Status of Infected Individuals by Scenario',
                       save = 'figures/scenario5_' + str(rr) + '-vaccine')

In [ ]:
# Scenario 6: Basic testing regime.

class TestingAndVaccineIntervention:
    def __init__(self, p_vaccinated, p_antibodies):
        self.pv = p_vaccinated
        self.pa = p_antibodies
        self.groups = defaultdict(list)
        self.group_counter = 0
    def testing(self, simulation, date): # Overload testing function to set vaccinations on first day.
        if date == datetime(2021, 9, 6):
            for i in simulation.semester.students:
                if random.random() <= self.pv:
                    simulation.state.V.add(i)
                if random.random() <= self.pa:
                    simulation.state.S.discard(i)
                    simulation.state.A.add(i)
                test_id = i % 16
                group_id = test_id % 4
                weekday = test_id // 4
                self.groups[(group_id, weekday)].append(i)
        
        # Each week we test one of the groups.  We divide it up by weekday.
        weekday = date.weekday()
        if weekday == 6:
            self.group_counter = (self.group_counter + 1) % 4
        for s in self.groups[(self.group_counter, weekday)]:
            if s in simulation.state.Ia or s in simulation.state.Is or \
                    s in simulation.state.Qa or s in simulation.state.Qs:
                simulation.schedule_positive_test_result(s)
    def __str__(self):
        return 'VaccineIntervention(Protection level' + str(self.pp) + ')'

scenario_sources = {}
scenario_distributions = {}
scenario_types = {}
ri, ro = 0.5, 0.5
for v in [.5, .6, .7, .8, .9, 1.0]:
    name = 'V' + str(v)
    ps = parameters_scenario3(v, ri, ro)
    ps.intervention_policy = TestingAndVaccineIntervention(v, 0.2)
    sources, distribution, types, summary = run_repetitions(
            None,
            ps,
            report = True,
            graphics = False)
    scenario_sources[name] = sources
    scenario_distributions[name] = distribution
    scenario_types[name] = types
    output = sources, distribution, types, summary
    pickle.dump(output, open('pickles/scenario6_' + name + '.p', 'wb'))

boxplot_totals(scenario_distributions, save = 'figures/scenario6')
boxplot_community(scenario_distributions, save = 'figures/scenario6')
stackedbar_sources(scenario_sources, 'Source of Infections by Scenario', 
                   save = 'figures/scenario6')
stackedbar_sources(scenario_types, 'Vaccination Status of Infected Individuals by Scenario',
                   save = 'figures/scenario6-vaccine')

In [ ]:
# Scenario 6b, c: Basic testing regime.

class TestingAndVaccineIntervention:
    def __init__(self, p_vaccinated, p_antibodies):
        self.pv = p_vaccinated
        self.pa = p_antibodies
        self.groups = defaultdict(list)
        self.group_counter = 0
    def testing(self, simulation, date): # Overload testing function to set vaccinations on first day.
        if date == datetime(2021, 9, 6):
            for i in simulation.semester.students:
                if random.random() <= self.pv:
                    simulation.state.V.add(i)
                if random.random() <= self.pa:
                    simulation.state.S.discard(i)
                    simulation.state.A.add(i)
                test_id = i % 16
                group_id = test_id % 4
                weekday = test_id // 4
                self.groups[(group_id, weekday)].append(i)
        
        # Each week we test one of the groups.  We divide it up by weekday.
        weekday = date.weekday()
        if weekday == 6:
            self.group_counter = (self.group_counter + 1) % 4
        for s in self.groups[(self.group_counter, weekday)]:
            if s in simulation.state.Ia or s in simulation.state.Is or \
                    s in simulation.state.Qa or s in simulation.state.Qs:
                simulation.schedule_positive_test_result(s)
    def __str__(self):
        return 'VaccineIntervention(Protection level' + str(self.pp) + ')'

for rr in [0.2, 0.8]:
    scenario_sources = {}
    scenario_distributions = {}
    scenario_types = {}
    ri, ro = rr, rr
    for v in [.5, .6, .7, .8, .9, 1.0]:
        name = '_' + str(rr) + '_V' + str(v)
        ps = parameters_scenario3(v, ri, ro)
        ps.intervention_policy = TestingAndVaccineIntervention(v, 0.2)
        sources, distribution, types, summary = run_repetitions(
                None,
                ps,
                report = True,
                graphics = False)
        scenario_sources[name] = sources
        scenario_distributions[name] = distribution
        scenario_types[name] = types
        output = sources, distribution, types, summary
        pickle.dump(output, open('pickles/scenario6' + name + '.p', 'wb'))

    boxplot_totals(scenario_distributions, save = 'figures/scenario6_' + str(rr))
    boxplot_community(scenario_distributions, save = 'figures/scenario6_' + str(rr))
    stackedbar_sources(scenario_sources, 'Source of Infections by Scenario', 
                       save = 'figures/scenario6_' + str(rr))
    stackedbar_sources(scenario_types, 'Vaccination Status of Infected Individuals by Scenario',
                       save = 'figures/scenario6_' + str(rr) + '-vaccine')